In [1]:
import pandas as pd
from gensim import corpora
from nltk import word_tokenize, pos_tag

In [2]:
df = pd.read_csv('/Users/geoffrey/jupyter/lesson/210131/test.csv')

In [3]:
df.head()

,titles,links,abstracts,meta_names,meta_keywords
0,A systematic review of the accuracy and effici...,https://pubmed.ncbi.nlm.nih.gov/31149604/,We are currently living in an era where the us...,"Lidia Galan-Lopez,Jorge Barcia-Gonzalez,Eliseo...","Clear aligners,Invisalign®,Invisible orthodont..."
1,Clustering of craniofacial patterns inKoreanch...,https://pubmed.ncbi.nlm.nih.gov/28670566/,The purpose of this study was to investigate w...,"Stephanie Maritza Anderson,Hoi-Jeong Lim,Ki-Be...","Adenotonsillar hypertrophy,Cluster analysis,Cr..."
2,The effect of western adaptation of Hispanic-A...,https://pubmed.ncbi.nlm.nih.gov/24511513/,To determine Korean facial profile preferences...,"Leo Toureno,Yoon-Ah Kook,Mohamed Bayome,Jae Hy...","Hispanic-American,Korean facial profile,Orthod..."
3,Predictors of midpalatal suture expansion by m...,https://pubmed.ncbi.nlm.nih.gov/31815104/,We sought to determine the predictors of midpa...,"Hyerin Shin,Chung-Ju Hwang,Kee-Joon Lee,Yoon J...","Adult treatment,Expansion,Miniscrew-assisted r..."
4,Part I. What drivesKoreanadults to seek orthod...,https://pubmed.ncbi.nlm.nih.gov/33144525/,To develop a standardized instrument to measur...,"Min-Hee Oh,Eun-A Kim,Ae-Hyun Park,MinSoo Kim,J...","Adult orthodontic treatment,Perception,Reliabi..."


## 예시 dataset (common_texts)를 가지고 간단한 예제 해보기

In [4]:
# 공식 홈페이지에서 가져온 예시 파일 실행 해보기
from gensim.test.utils import common_texts
from gensim.models.word2vec import Word2Vec

In [5]:
common_texts

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

In [6]:
model = Word2Vec(sentences=common_texts, window=2, min_count=1, workers=4)
# model.save("word2vec.model")


In [7]:
# 특정 단어에 대한 임베딩 된 벡터를 알고 싶을 때 사용 (wv는 word vector를 뜻함)
word = 'human'
model.wv[word]

array([ 4.9559469e-04, -1.4498014e-03,  4.6622702e-03, -2.1706307e-03,
       -3.0431170e-03,  1.8999146e-03, -2.5196711e-03, -2.4258308e-03,
       -1.5195345e-03,  4.7205016e-03,  3.9616427e-03, -6.2622700e-04,
       -1.8111294e-03,  4.8764434e-04, -2.1511563e-03,  2.3970113e-03,
        1.3064079e-03, -3.3367134e-03, -4.7804648e-03,  4.1177040e-03,
        1.1285308e-03,  2.2603872e-03, -4.6747942e-03,  2.5804625e-03,
        2.5342659e-03,  2.1632533e-03,  1.4864703e-03,  1.3405244e-03,
       -1.5445962e-03,  1.7269537e-03, -1.5392802e-03, -4.0344349e-03,
        3.5879689e-03,  1.5678587e-03,  4.9294345e-03,  2.4081110e-03,
       -1.6220700e-03,  4.5378148e-03,  4.9475078e-03, -4.6175168e-04,
       -1.6110055e-03,  4.8746602e-03,  2.6936231e-03,  1.7647655e-03,
       -1.4865083e-04,  6.3788245e-04, -3.7781508e-03, -2.7259251e-03,
        1.0313580e-04, -3.4938238e-03,  2.6551550e-03, -3.1526589e-03,
        1.2766829e-03,  3.8229057e-03,  2.6333251e-03,  3.1885197e-03,
      

In [8]:
# 특정 단어 2개에 대한 cosine similarity를 구하고 싶을 때 사용
model.wv.similarity('minors', 'trees')

0.03228431

In [9]:
# 가장 가까운 단어들 나열
model.wv.most_similar(positive=["human"])

[('response', 0.07711795717477798),
 ('computer', 0.030471235513687134),
 ('interface', 0.020529383793473244),
 ('survey', 0.012123361229896545),
 ('time', 0.004509247839450836),
 ('trees', -0.023135080933570862),
 ('graph', -0.034140028059482574),
 ('eps', -0.06613792479038239),
 ('system', -0.08018376678228378),
 ('user', -0.09784466028213501)]

## 실제 abstracts를 가지고 진행해보기

In [10]:
# tokenize 복습!
# tokenize : 문장을 token (학습에 사용되는 최소 단위) 으로 쪼개는 작업
df.abstracts[0]

'We are currently living in an era where the use of computer-aided design/computer-aided manufacturing has allowed individualized orthodontic treatments, but has also incorporated enhanced digitalized technology that does not permit improvisation. The purpose of this systematic review was to analyze publications that assessed the accuracy and efficiency of the Invisalign® system. A systematic review was performed using a search strategy to identify articles that referenced Invisalign®, which were published between August 2007 and August 2017, and listed in the following databases: MEDLINE, Embase, Cochrane Library, Web of Knowledge, Google Scholar, and LILACS. Additionally, a manual search of clinical trials was performed in scientific journals and other databases. To rate the methodological quality of the articles, a grading system described by the Swedish Council on Technology Assessment in Health Care was used, in combination with the Cochrane tool for risk of bias assessment. We se

In [11]:
test = df.abstracts[0]
print(word_tokenize(test))

['We', 'are', 'currently', 'living', 'in', 'an', 'era', 'where', 'the', 'use', 'of', 'computer-aided', 'design/computer-aided', 'manufacturing', 'has', 'allowed', 'individualized', 'orthodontic', 'treatments', ',', 'but', 'has', 'also', 'incorporated', 'enhanced', 'digitalized', 'technology', 'that', 'does', 'not', 'permit', 'improvisation', '.', 'The', 'purpose', 'of', 'this', 'systematic', 'review', 'was', 'to', 'analyze', 'publications', 'that', 'assessed', 'the', 'accuracy', 'and', 'efficiency', 'of', 'the', 'Invisalign®', 'system', '.', 'A', 'systematic', 'review', 'was', 'performed', 'using', 'a', 'search', 'strategy', 'to', 'identify', 'articles', 'that', 'referenced', 'Invisalign®', ',', 'which', 'were', 'published', 'between', 'August', '2007', 'and', 'August', '2017', ',', 'and', 'listed', 'in', 'the', 'following', 'databases', ':', 'MEDLINE', ',', 'Embase', ',', 'Cochrane', 'Library', ',', 'Web', 'of', 'Knowledge', ',', 'Google', 'Scholar', ',', 'and', 'LILACS', '.', 'Additi

In [12]:
df.abstracts

0      We are currently living in an era where the us...
1      The purpose of this study was to investigate w...
2      To determine Korean facial profile preferences...
3      We sought to determine the predictors of midpa...
4      To develop a standardized instrument to measur...
                             ...                        
358    The purpose of this study was to determine the...
359    This study investigated whether temporomandibu...
360    A Brazilian girl aged 14 years and 9 months pr...
361    To evaluate lower incisor position and bony su...
362    A 19-year-old woman presented to our dental cl...
Name: abstracts, Length: 363, dtype: object

In [13]:
# series.map(function) -> 시리즈의 row에 함수를 전부 적용한다

def tokenizing(x):
    x = word_tokenize(x)
    return x

df.abstracts = df.abstracts.map(tokenizing)

In [14]:
df.abstracts

0      [We, are, currently, living, in, an, era, wher...
1      [The, purpose, of, this, study, was, to, inves...
2      [To, determine, Korean, facial, profile, prefe...
3      [We, sought, to, determine, the, predictors, o...
4      [To, develop, a, standardized, instrument, to,...
                             ...                        
358    [The, purpose, of, this, study, was, to, deter...
359    [This, study, investigated, whether, temporoma...
360    [A, Brazilian, girl, aged, 14, years, and, 9, ...
361    [To, evaluate, lower, incisor, position, and, ...
362    [A, 19-year-old, woman, presented, to, our, de...
Name: abstracts, Length: 363, dtype: object

In [15]:
# pandas series를 list로 바꾸는 방법 : series.tolist()
docs = df.abstracts.tolist()
print(docs[:2])

[['We', 'are', 'currently', 'living', 'in', 'an', 'era', 'where', 'the', 'use', 'of', 'computer-aided', 'design/computer-aided', 'manufacturing', 'has', 'allowed', 'individualized', 'orthodontic', 'treatments', ',', 'but', 'has', 'also', 'incorporated', 'enhanced', 'digitalized', 'technology', 'that', 'does', 'not', 'permit', 'improvisation', '.', 'The', 'purpose', 'of', 'this', 'systematic', 'review', 'was', 'to', 'analyze', 'publications', 'that', 'assessed', 'the', 'accuracy', 'and', 'efficiency', 'of', 'the', 'Invisalign®', 'system', '.', 'A', 'systematic', 'review', 'was', 'performed', 'using', 'a', 'search', 'strategy', 'to', 'identify', 'articles', 'that', 'referenced', 'Invisalign®', ',', 'which', 'were', 'published', 'between', 'August', '2007', 'and', 'August', '2017', ',', 'and', 'listed', 'in', 'the', 'following', 'databases', ':', 'MEDLINE', ',', 'Embase', ',', 'Cochrane', 'Library', ',', 'Web', 'of', 'Knowledge', ',', 'Google', 'Scholar', ',', 'and', 'LILACS', '.', 'Addit

In [19]:
# word2vec 학습
model = Word2Vec(sentences=docs,
                 window=5, # window size (양옆 어느정도 거리까지 학습할 것인지)
                 min_count=5, # 최소 이정도는 등장해야 학습에 포함시킴
                 workers=4, # 사용할 cpu threads
                 sg=1, # 1이면 skip-gram, 0이면 C-BOW
                 size=256,
                 iter=10,
                )

# vector_size, epochs 하이퍼파라미터 안먹는 이유 알아보기 

In [84]:
model.wv.most_similar(positive=["Cochrane"])

[('categories', 0.9972798824310303),
 ('differentiation', 0.9972244501113892),
 ('APH', 0.9968062043190002),
 ('mini-screw', 0.9967907667160034),
 ('investigation', 0.9967794418334961),
 ('manufacturers', 0.9967294931411743),
 ('original', 0.996656060218811),
 ('ankylosis', 0.9965190887451172),
 ('composition', 0.9964578151702881),
 ('osseous', 0.996235728263855)]